# Goal:
Create library, in the best case with single-call functionality.
Task of the lib: Do all preprocessing steps and provide data to fit a keras NN
Should be basically an adapter function which also includes some preprocessing steps

This file is for dev purpose, the final version musst be stored in a .py file!

## Limitation:
I am always predicting all features. Which of them are really of interest will be decided in the evaluation part!


In [1]:
# univariate data preparation
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

import pandas as pd
import tensorflow as tf
import myLibrary as mL
from sklearn.metrics import mean_squared_error, mean_absolute_error

import netCDF4 as nc
import matplotlib.pyplot as plt

In [2]:
data = mL.get_data(
    stations=["42036", "41117"],
    years=["2020", "2021"],
    nan_threshold=0.8,
    features=["PRES","ATMP", "WTMP", "DEWP"],
    #features=None,
    ERA5=False
    )

data

Started with  2020 . Previous year took:   0.0006608963012695312 seconds
from disc
from disc
Started with  2021 . Previous year took:   4.141801834106445 seconds
from disc
from disc
Finished downloading - now merging it together!


,PRES_42036,ATMP_42036,WTMP_42036,DEWP_42036,ATMP_41117,WTMP_41117
timestamp,,,,,,
2020-01-01 00:00:00,1019.5,17.2,21.8,6.8,26.7,17.4
2020-01-01 01:00:00,1019.8,17.3,21.8,7.3,26.7,17.4
2020-01-01 02:00:00,1020.1,17.8,21.8,7.6,26.7,17.4
2020-01-01 03:00:00,1020.7,18.1,21.8,8.4,26.7,17.4
2020-01-01 04:00:00,1020.7,18.2,21.8,7.9,26.7,17.4
...,...,...,...,...,...,...
2021-12-31 19:00:00,1016.0,23.5,22.8,23.5,21.3,20.5
2021-12-31 20:00:00,1015.8,23.5,22.9,23.5,21.7,20.6
2021-12-31 21:00:00,1015.8,23.6,22.9,23.6,21.7,20.7


In [8]:
data.isna().sum().sum()

0

# Data to stationary

In [3]:
def data_to_stationary(data: pd.DataFrame, n: int =1):
    data_stationary = pd.DataFrame()

    for col in data.columns:
        data_stationary[col] = data[col] - data[col].shift(n)  # y = value(i) - value(i-n)

    data_stationary = data_stationary.iloc[n:]  # remove first n entries since there is no delta value for them
    return data_stationary

data_stationary = data_to_stationary(data)
data_stationary

,PRES_42036,ATMP_42036,WTMP_42036,DEWP_42036,ATMP_41117,WTMP_41117
timestamp,,,,,,
2020-01-01 01:00:00,0.3,0.1,0.0,0.5,0.0,0.0
2020-01-01 02:00:00,0.3,0.5,0.0,0.3,0.0,0.0
2020-01-01 03:00:00,0.6,0.3,0.0,0.8,0.0,0.0
2020-01-01 04:00:00,0.0,0.1,0.0,-0.5,0.0,0.0
2020-01-01 05:00:00,-0.5,0.1,0.0,0.4,0.0,0.0
...,...,...,...,...,...,...
2021-12-31 19:00:00,-0.6,0.1,0.1,0.1,0.2,-0.1
2021-12-31 20:00:00,-0.2,0.0,0.1,0.0,0.4,0.1
2021-12-31 21:00:00,0.0,0.1,0.0,0.1,0.0,0.1
